In [ ]:
import numpy as np
import tensorflow as tf
import shutil
import os
import pandas as pd

In [ ]:
Training_Dir = 'train'

In [5]:
def count_labels(data_dir):
    classes_path = os.path.join(data_dir, '_classes.csv')
    df = pd.read_csv(classes_path)
    label_counts = df.iloc[:, 1:].sum(axis=0)

    return label_counts

In [6]:
# Menghitung dan menampilkan jumlah gambar untuk masing-masing label
train_label_counts = count_labels(Training_Dir)

print("Jumlah gambar untuk masing-masing label di training set:")
print(train_label_counts)

Jumlah gambar untuk masing-masing label di training set:
Acne            2105
Blackheads        72
Dark Spots       190
Dry Skin         326
Eye bags         156
Normal Skin      178
Oily Skin        180
Pores            197
Skin Redness      33
Wrinkles         297
dtype: int64


In [ ]:
def add_new_images_to_training(new_images_dir, training_dir, csv_path):
    # Try to read the existing CSV file
    try:
        df = pd.read_csv(csv_path)
    except PermissionError:
        print(f"Permission denied: {csv_path}. Make sure the file is not open in another application and you have sufficient permissions.")
        return

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # List of valid labels
    valid_labels = ['Acne', 'Blackheads', 'Dark Spots', 'Dry Skin', 'Eye bags', 
                    'Normal Skin', 'Oily Skin', 'Pores', 'Skin Redness', 'Wrinkles']

    # List to store new entries
    new_entries = []

    # Loop through each label directory
    for label in valid_labels:
        label_dir = os.path.join(new_images_dir, label)

        # Check if it's a directory and contains images
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)

                # Check if the image already exists in the training folder
                if os.path.isfile(image_path) and image_name not in df['filename'].values:
                    # Define the new path in the training folder
                    new_image_path = os.path.join(training_dir, image_name)

                    # Copy the image to the training folder
                    shutil.copy(image_path, new_image_path)

                    # Create a new entry for the CSV
                    new_entry = {'filename': image_name}
                    # Set the label for the current image
                    for col in valid_labels:
                        new_entry[col] = 1 if col == label else 0  # Set label

                    # Add the new entry to the list
                    new_entries.append(new_entry)

    # If there are new entries, combine them with the existing DataFrame
    if new_entries:
        new_df = pd.DataFrame(new_entries)
        
        # Ensure the columns in new_df match those in df
        new_df = new_df[df.columns]  # Reorder columns to match df
        
        # Combine the DataFrames
        df = pd.concat([df, new_df], ignore_index=True)

    # Try to save the updated DataFrame back to the CSV file
    try:
        df.to_csv(csv_path, index=False)
        print("New images have been added and the CSV has been updated.")
    except PermissionError:
        print(f"Permission denied: {csv_path}. Make sure the file is not open in another application and you have sufficient permissions.")

In [8]:
new_images_directory = 'New dataset'  # Ganti dengan path ke folder gambar baru
training_directory = 'train'  # Folder training
csv_file_path = os.path.join(training_directory, '_classes.csv')  # Path ke file CSV

add_new_images_to_training(new_images_directory, training_directory, csv_file_path)

New images have been added and the CSV has been updated.


In [9]:
update_train_label_counts = count_labels(Training_Dir)

print("Jumlah gambar untuk masing-masing label di training set:")
print(update_train_label_counts)

Jumlah gambar untuk masing-masing label di training set:
Acne            2105
Blackheads        72
Dark Spots       404
Dry Skin         351
Eye bags         194
Normal Skin      341
Oily Skin        326
Pores            197
Skin Redness     601
Wrinkles         537
dtype: int64
